In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# ========== 1. 准备无监督数据 ==========
# 读取CSV文件
data = pd.read_csv('your_data.csv')
X = data.values  # 如果整个文件都是数值特征
# 或者选择特定列: X = data[['col1', 'col2', 'col3']].values

# 数据预处理（根据需要）
# 1. 处理缺失值
# X = pd.DataFrame(X).fillna(method='mean').values
# 2. 标准化（可选）
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

n_samples, n_features = X.shape
print(f'数据形状: {n_samples} 样本, {n_features} 特征')

# ========== 2. 构造伪标签数据（用于无监督随机森林） ==========
# 随机打乱数据并构造伪造样本
X_fake = np.random.uniform(low=X.min(axis=0), high=X.max(axis=0), size=X.shape)

# 合并真实和伪造数据
X_combined = np.vstack((X, X_fake))
y_combined = np.hstack((np.ones(n_samples), np.zeros(n_samples)))  # 1: 真实，0: 伪造

# ========== 3. 训练随机森林分类器 ==========
rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf.fit(X_combined, y_combined)

# ========== 4. 提取 Gini 不纯度减少量 ==========
feature_importance_raw = rf.feature_importances_  # 即 MDG

# ========== 5. 标准化 MDG 值到 [0,1] 区间 ==========
scaler = MinMaxScaler()
feature_importance_scaled = scaler.fit_transform(feature_importance_raw.reshape(-1, 1)).flatten()

# ========== 6. 输出结果 ==========
# 生成特征名称（如果数据没有列名）
# 如果使用pandas读取的数据有列名，可以用: feature_names = data.columns.tolist()
try:
    feature_names = load_iris().feature_names  # 示例数据的特征名
except:
    feature_names = [f'Feature_{i+1}' for i in range(n_features)]  # 自动生成特征名

# 创建结果DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'MDG': feature_importance_raw,
    'MDG_scaled': feature_importance_scaled,
    'Rank': range(1, len(feature_importance_scaled) + 1)
}).sort_values(by='MDG_scaled', ascending=False)

# 重新设置排名
importance_df['Rank'] = range(1, len(importance_df) + 1)

print('特征重要性排序结果:')
print(importance_df)
